In [58]:
#Testing if Data Retrieval is functional from AWS 

import pandas as pd
import s3fs


fs = s3fs.S3FileSystem(anon=True)


directory_path = 's3://aws-public-blockchain/v1.0/eth/token_transfers/date=2017-01-01'


files = fs.ls(directory_path)



parquet_files = [file for file in files if file.endswith('.parquet')]


file_path = 's3://' + parquet_files[0]

print(parquet_files[:5])

with fs.open(file_path, mode='rb') as f:
    df = pd.read_parquet(f, engine='pyarrow')


print(len(df))


['aws-public-blockchain/v1.0/eth/token_transfers/date=2017-01-01/part-00000-77bd61b1-dc0a-40fb-a715-042fc265ea2e-c000.snappy.parquet']
489


In [60]:
#Exploring the Etherium dataset
import s3fs


fs = s3fs.S3FileSystem(anon=True)


directory_path = 's3://aws-public-blockchain/v1.0/eth'


files = fs.ls(directory_path)


parquet_files = [file for file in files]

print(parquet_files[:5])


['aws-public-blockchain/v1.0/eth/blocks', 'aws-public-blockchain/v1.0/eth/contracts', 'aws-public-blockchain/v1.0/eth/logs', 'aws-public-blockchain/v1.0/eth/token_transfers', 'aws-public-blockchain/v1.0/eth/traces']


In [ ]:
!pip3 install ethereum-etl

In [4]:
#Testing data loading for one day
import pandas as pd
import s3fs
from bigchaindb_driver import BigchainDB
from bigchaindb_driver.crypto import generate_keypair


bdb = BigchainDB('http://localhost:9984')
user = generate_keypair()


fs = s3fs.S3FileSystem(anon=True)
directory_path = 's3://aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-27'
files = fs.ls(directory_path)
parquet_files = [file for file in files if file.endswith('.parquet')]
file_path = 's3://' + parquet_files[0]

with fs.open(file_path, mode='rb') as f:
    df = pd.read_parquet(f, engine='pyarrow')


df['block_timestamp'] = df['block_timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')


for index, row in df.iterrows():
    asset_data = {
        'token_address': row['token_address'],
        'from_address': row['from_address'],
        'to_address': row['to_address'],
        'value': row['value'],
        'transaction_hash': row['transaction_hash'],
        'log_index': row['log_index'],
        'block_number': row['block_number'],
        'block_hash': row['block_hash'],
        'block_timestamp': row['block_timestamp'],  # Now a string
    }
    
    prepared_tx = bdb.transactions.prepare(
        operation='CREATE',
        signers=user.public_key,
        asset={'data': asset_data},
    )
    
    fulfilled_tx = bdb.transactions.fulfill(
        prepared_tx,
        private_keys=user.private_key,
    )
    
    sent_tx = bdb.transactions.send_commit(fulfilled_tx)
    print(f"Transaction {index+1} committed: {sent_tx['id']}")

print("All transactions have been processed.")


Transaction 1 committed: f39965333bff13bcd3656e3d39ad844d0ab9cf1c6886926c1c56f96e2094d835
Transaction 2 committed: 0ae19128ff63831947c81993d363afdc2a5647dba7c5a5444ba9f8494970042e
Transaction 3 committed: f1b0b14f6de056b1008a62993774022b5a5c9ed9603a0ebdeee57310fb4b6314
Transaction 4 committed: 009978f50c133bf293c29eb5148a7a56fe377653b151f3587966d77008348b38
Transaction 5 committed: 0d50b6e52d171d6e4853615f08aebf134c9492d398d1cfd203874ba683cf82b0
Transaction 6 committed: 11a10debfd7fd660bea1ef25e0fb53efac7d60af64b4f8794c56daeb558a6cea
Transaction 7 committed: 50ea85b0cff6e5fb070ea97113c6118c433da03cb1d57f75ea6d2ce63bbd89d5
Transaction 8 committed: ad1382dc0ef21a8548e31a544a06baa50dd6bdc0df072b63f254052da5e513a3
All transactions have been processed.


In [ ]:
from bigchaindb_driver import BigchainDB


bdb = BigchainDB('http://localhost:9984')


transaction_id = '804ed22a41762d97897452366eba9e049b9e2d54246c0535094adad397d7944a'


retrieved_tx = bdb.transactions.retrieve(transaction_id)


print(retrieved_tx)


{'inputs': [{'owners_before': ['HRoT8xypfGZ8NKRYEVVKZAntTVMX8sr9dZi2kX66UwcS'], 'fulfills': None, 'fulfillment': 'pGSAIPQVc8AUDMnBFrNJFXG9IsvaKbxj9DsAxzuC15_aACdHgUABqfc08ZkCAqE35-HxyUyaM_biP7LovFscbdnFVaL3Dazi4NlaVCQNHx0rn3zLZxZsQoiX11zGiGvYDqhGYEQI'}], 'outputs': [{'public_keys': ['HRoT8xypfGZ8NKRYEVVKZAntTVMX8sr9dZi2kX66UwcS'], 'condition': {'details': {'type': 'ed25519-sha-256', 'public_key': 'HRoT8xypfGZ8NKRYEVVKZAntTVMX8sr9dZi2kX66UwcS'}, 'uri': 'ni:///sha-256;bnkcseR_t-9JT4E_Fqlr85Gk8grE32TuVSrKHFQEjTM?fpt=ed25519-sha-256&cost=131072'}, 'amount': '1'}], 'operation': 'CREATE', 'metadata': None, 'asset': {'data': {'token_address': '0x27cb40ce7eb4d078196923d608eb903a17e0c0ed', 'from_address': '0x00000000000000000000000097e58c7d37cba1a1e2ecbb2a5b23f8d127b6892d', 'to_address': '0x000000000000000000000000b1a2b43a7433dd150bb82227ed519cd6b142d382', 'value': 1.1626938841057844e+202, 'transaction_hash': '0x30a37b6b236b9abe85801e46f297b7f75f234173503af47d3c48b15691933e33', 'log_index': 1, 

In [6]:
#Transforming the data to make it suitable for loading in the BigchainDB
import pandas as pd
import s3fs
from bigchaindb_driver import BigchainDB
from bigchaindb_driver.crypto import generate_keypair


def load_data_from_parquet(file_path, bdb, user):
    with fs.open(file_path, mode='rb') as f:
        df = pd.read_parquet(f, engine='pyarrow')


    df['block_timestamp'] = df['block_timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')


    for index, row in df.iterrows():
        asset_data = {
            'token_address': row['token_address'],
            'from_address': row['from_address'],
            'to_address': row['to_address'],
            'value': row['value'],
            'transaction_hash': row['transaction_hash'],
            'log_index': row['log_index'],
            'block_number': row['block_number'],
            'block_hash': row['block_hash'],
            'block_timestamp': row['block_timestamp'],  # Now a string
        }

        prepared_tx = bdb.transactions.prepare(
            operation='CREATE',
            signers=user.public_key,
            asset={'data': asset_data},
        )

        fulfilled_tx = bdb.transactions.fulfill(
            prepared_tx,
            private_keys=user.private_key,
        )

        sent_tx = bdb.transactions.send_commit(fulfilled_tx)
        print(f"Transaction {index+1} committed: {sent_tx['id']}")


bdb = BigchainDB('http://localhost:9984')
user = generate_keypair()


fs = s3fs.S3FileSystem(anon=True)


directories = [
    'aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-27',
    'aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-30',
    'aws-public-blockchain/v1.0/eth/token_transfers/date=2015-11-02',
    'aws-public-blockchain/v1.0/eth/token_transfers/date=2015-11-03',
    'aws-public-blockchain/v1.0/eth/token_transfers/date=2015-11-04',
]


for directory_path in directories:
    files = fs.ls(directory_path)
    parquet_files = [file for file in files if file.endswith('.parquet')]

    for file_path in parquet_files:
        full_file_path = 's3://' + file_path
        print(f"Processing file: {full_file_path}")
        load_data_from_parquet(full_file_path, bdb, user)

print("All transactions from specified dates have been processed.")


Processing file: s3://aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-27/part-00000-2ad0be30-0be0-4d1a-8d1f-c7240a656db5-c000.snappy.parquet
Transaction 1 committed: 6e3377acced53f78e9542143f1fdc0ae295f935f980e00cad28ef37d47a4eede
Transaction 2 committed: 5c53c0e6eaa70f908080d36f53f0652561fd85bca715e52589be6e7e7c47960a
Transaction 3 committed: fbcfd19cf190d5d7de5556445cbb6519cd36ffae511d50485b55f8856841bfe3
Transaction 4 committed: b00be8cdc4f86cb36f4be823d479db7ce81dcf55836dbd3db8bcc3943b839890
Transaction 5 committed: cafacef1692ee921700bbb909b5099e1fb1b5ffdde93628cd4d3b3fdf22767db
Transaction 6 committed: a99d8ae882d3b1f33a6b7f9714dbe8aa4a28080bfcb47d6a10b1016bc51b53fa
Transaction 7 committed: f0019ebc7f734b440d48c051cba10706053b016a7086fd3e892b244e7ca58e43
Transaction 8 committed: 31b341e8f4595b4c40b663d958f3f9cf4531172778100beb28fbe100a64ad570
Processing file: s3://aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-30/part-00000-7996c555-d36b-4163-b792-77c5afe

In [7]:
query = '0x27cb40ce7eb4d078196923d608eb903a17e0c0ed'
assets = bdb.assets.get(search=query)
print(assets)


[{'data': {'token_address': '0x27cb40ce7eb4d078196923d608eb903a17e0c0ed', 'from_address': '0x00000000000000000000000097e58c7d37cba1a1e2ecbb2a5b23f8d127b6892d', 'to_address': '0x000000000000000000000000b1a2b43a7433dd150bb82227ed519cd6b142d382', 'value': 1.1626938841057844e+202, 'transaction_hash': '0x30a37b6b236b9abe85801e46f297b7f75f234173503af47d3c48b15691933e33', 'log_index': 1, 'block_number': 448028, 'block_hash': '0x6df73b144364716f28a0ea926e77832221c05b0737209e831339f163e97ed7ad', 'block_timestamp': '2015-10-27T14:08:01'}, 'id': '804ed22a41762d97897452366eba9e049b9e2d54246c0535094adad397d7944a'}, {'data': {'token_address': '0x27cb40ce7eb4d078196923d608eb903a17e0c0ed', 'from_address': '0x00000000000000000000000097e58c7d37cba1a1e2ecbb2a5b23f8d127b6892d', 'to_address': '0x0000000000000000000000009b22a80d5c7b3374a05b446081f97d0a34079e7f', 'value': 1.1626938841057844e+202, 'transaction_hash': '0x99d443aa6f49e69d83dec151d1607a52d3a77e59b3ec7a36ad8530de6a5c979c', 'log_index': 1, 'block_

In [14]:
!pip install psutil

     |████████████████████████████████| 248 kB 139 kB/s eta 0:00:01


In [55]:
import s3fs
import pandas as pd
import psutil
import time
from datetime import datetime, timedelta
from bigchaindb_driver import BigchainDB
from bigchaindb_driver.crypto import generate_keypair

# BigchainDB setup
bdb = BigchainDB('http://localhost:9984')
user = generate_keypair()

# S3 setup
fs = s3fs.S3FileSystem(anon=True)
base_directory_path = 's3://aws-public-blockchain/v1.0/eth/token_transfers/date='

# Helper function to monitor and log system resources
def log_system_resources():
    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().percent
    print(f"CPU Usage: {cpu_usage}%, Memory Usage: {memory_usage}%")

def load_data_into_bigchaindb(dataframe):
    # Convert Timestamp columns to strings in ISO 8601 format
    for col in dataframe.select_dtypes(include=['datetime64']):
        dataframe[col] = dataframe[col].dt.strftime('%Y-%m-%dT%H:%M:%S')
    
    # Replace NaN, Infinity, and -Infinity with None (which becomes 'null' in JSON)
    dataframe = dataframe.replace([float('inf'), float('-inf'), float('nan')], None)

    for index, row in dataframe.iterrows():
        # Using the hash as the only piece of data for the asset
        asset_data = {
            'hash': row['block_hash']
        }
        
        # Preparing metadata from the rest of the row, excluding the hash
        metadata = row.drop('block_hash').to_dict()
        metadata = {k: (None if pd.isna(v) else v) for k, v in metadata.items()}
        
        prepared_tx = bdb.transactions.prepare(
            operation='CREATE',
            signers=user.public_key,
            asset={'data': asset_data},
            metadata=metadata,
        )
        fulfilled_tx = bdb.transactions.fulfill(
            prepared_tx,
            private_keys=user.private_key,
        )
        sent_tx = bdb.transactions.send_commit(fulfilled_tx)
        print(f"Transaction {index+1} committed: {sent_tx['id']}")


def process_date_range(start_date, end_date):
    start_time = time.time()
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime('%Y-%m-%d')
        directory_path = f'{base_directory_path}{date_str}'
        try:
            files = fs.ls(directory_path)
            parquet_files = [file for file in files if file.endswith('.snappy.parquet')]
            for file_path in parquet_files:
                with fs.open(f's3://{file_path}', mode='rb') as f:
                    df = pd.read_parquet(f, engine='pyarrow')
                    load_data_into_bigchaindb(df)
        except Exception as e:
            print(f"Error processing {date_str}: {e}")
        current_date += timedelta(days=1)
        log_system_resources()  # Log system resources after each day's data is processed

    end_time = time.time()
    print(f"Total time for processing range: {end_time - start_time} seconds")

start_date = datetime(2020, 1, 1)
process_date_range(start_date, start_date + timedelta(days=5000))  # 5000 records 
# process_date_range(start_date, start_date + timedelta(days=10000))  # 10000 records
# process_date_range(start_date, start_date + timedelta(days=100000))  # 100000 records
# process_date_range(start_date, start_date + timedelta(days=1000000))  # 1000000 records



Transaction 1 committed: b903ebf94e31acf6beee624b68dd1dbf17234a0b16920992d20ecb29e5085222
Transaction 2 committed: fba1ce7dca7c7e979bb9378f6f5480097eccb2fafc9ce7018b739b98f8ff053b
Transaction 3 committed: a9d49eaf7248d18dda9b115d2758d1c5d4ff7dc72d409bbbbd860e7059c5b8e7
Transaction 4 committed: 89df897afa76b26f54dcbbc9360a48420e024da030cb0d93c70dc75485f1d515
Transaction 5 committed: 1fbb3c739c2510b1dd65c75dcb7ea12f6bf1b1711ecc06b6fd51f0f816b8d93b
Transaction 6 committed: 66b8435f8e44ffc047a871a28c5ddb2dc5d1de71747a245c4bd16b44effee9de
Transaction 7 committed: 1f09d1bde1190f49138bd5f7365cc1f0c4754be044ac25224449bee048daa5c7
Transaction 8 committed: ba8105e3e3ed82674faaf9f39adac934633f53b2068d3cf11568f2e140d799d1
Transaction 9 committed: bb6cb63030a3720606a4f116e1adfed05539cf4a344c0a186a43bf9d1294d19f
Transaction 10 committed: f1073df5d0d1b4bf78c7a819d56394ebad76a55b8748c06fdf4cc3d389fed16f
Transaction 11 committed: 448386c5f3bdeec0865d63c5feb2dcbd4c32ae728766079ee47f118dedb067dc
Transact

Transaction 92 committed: 482c6f75fd60460aa11509bffd2653104082405043e91ac9dcf49aad04f41e07
Transaction 93 committed: 80541278760ef7ad9dba80ae5eada5d259ea8988e7a8505c4b013c45e9077024
Transaction 94 committed: e77e3f17efce4a4df4515e193f6648b7af6e056e97057a87ba950f4c88e3a928
Transaction 95 committed: a0438066cde8f5ba6eb9631dce2ca37cbf534b991afea6f72e7d33b3eba5f3b9
Transaction 96 committed: 1902a8834c25f7afe77c320a0254f18f10225610223c6f9f278d9d17b39d26d2
Transaction 97 committed: d42b438206abee0b138bc86e23270b1ec6c697f60dfe2ef850ce782f7659f2e2
Transaction 98 committed: 640f481a57f255d6353973efbc5efb6278f4b7dba1d40cc6749bff2ed13f785b
Transaction 99 committed: 51d1e633aa5628f44a333bddd23e006a3f778bc4b4e0ae7a02eefb89d87ec451
Transaction 100 committed: f6566422cc7d724ad759b5c0d0fd2179b394dfce537062a57e7475f1988cb06c
Transaction 101 committed: e83b238c29b3dc03303e502f8b0f302d351143a6f83eeccf6592c91baa241555
Transaction 102 committed: 73423f8dbf060945e19821a0af3b1f29c2785ea26da907756280718073194

Transaction 182 committed: fed9aba289b6e24066008159524117247246a2a5bc5cdff03d1ee63011cbd357
Transaction 183 committed: f4ff8c3a7c739e4aecb9b2dacf121343dea5d30dc3213389250d51e6794f3c3d
Transaction 184 committed: f0f86bc7f4f3913dd4e628781fd043802dfac2e21f2aa27b74c7555f96eba0d0
Transaction 185 committed: 8cbfe21d95c74e4b4c1b9d45e4ed25614df227392a29463fadb2588175bc669b
Transaction 186 committed: ef000b62b13489a992bb61f2694d36bb17faace77db73e3a1b211f6d34317184
Transaction 187 committed: 3e2d5bb414a4552442fc48735f52cd669f1597cb9b6f973a2591d1ddbf630462
Transaction 188 committed: 128e14254e5d7ebb7eff9d9a4a2146185fd96e9ee72760e88548f87257632f6f
Transaction 189 committed: dcb68b928f5113f4a3ae7c3f460b4ecd3faece69a42cf3ae18340263f9ed88b5
Transaction 190 committed: dfd64b77bd11f50b23a57d62d35b9699de9283d36c4c61d449ba5545b68e5795
Transaction 191 committed: a4065ad14d83acbbe87019dff9b8db0e7df78bf2eb1fc515249d9fbcdf1f7506
Transaction 192 committed: e1cfb438c9db31e9a6ef772294ba194038f1547c094bf07351057

Transaction 272 committed: 8f0cc6a1c5e64da84b65b3bc0bb34a8a4996e805af9f35cfa1e3ed0be329e655
Transaction 273 committed: 9057583531e608c80cc1520a8f794c6af16d827cb4172a7c0c704c650aa8dade
Transaction 274 committed: adb94a3853122c63216c116f01515579b8884b86764453ded318b7db7bca0499
Transaction 275 committed: 2c26e2d644cfb86bfa5b7e28e86bdc9910e5b4aa280d31a740294d5eadee9295
Transaction 276 committed: acbd400cb1cd5fa2d476122839aa1ff287b827f1540fdea4eb2a5806d56966da
Transaction 277 committed: 0d80ff1d0ff5fa53cecf1f2ed6b6e9329c1e9a6b85055800ddf30a699e329ce3
Transaction 278 committed: 28ba072a0257d9787632194c28c94f2504925509ddcadefd12f721356b3d1c90
Transaction 279 committed: d5ab904c87c5f75ca09fefd80c81076b30f4cb02a6cfb6d28e5bf9111d720a2c
Transaction 280 committed: 12a396f2fc3704ef985ac8d4a7af4a99c0711260ea14710667ec69cbe8baa7d7
Transaction 281 committed: 25855a6cef35d1c679cb6156fe74eee382e402e9eba897c61594046f2d8db853
Transaction 282 committed: f0177534c9384518e02e30ccb7d1aa9afb4722a945e14465ac8ce

Transaction 363 committed: 67a6926cc827368c93c3bd287cfd70ffc30cb2d33334fcff9e5bd4f8b012505f
Transaction 364 committed: 5a87810d458e11dafbc1a0f7f7fb5dffa838cb202147b923f8df173927ed047a
Transaction 365 committed: e08ef6aa5de52a08e88a535bc6ffc351a9c75ba3500aca744c3d37f27248ef8b
Transaction 366 committed: 0e968aecadb6d8c19b5acb2777ad482f3ee1439fa88834c56c489e20096c6697
Transaction 367 committed: 6a63196193d82f87da33d3472ce4155975ee8a9bfcbc67967cc46d7c45855c60
Transaction 368 committed: 5d3acb8e4aee47f1c572161558ec8137cd71570115e397a43b241b37865e1212
Transaction 369 committed: b9249dbd5c08f1e6bc9d39285deff8c444c462c50c2ae6c9a0ed5b590387ccb9
Transaction 370 committed: 97d45f3a7c30a6c2fb41aa4d4193ae575db679b29f1b4d721a5f4bb3fb5741dd
Transaction 371 committed: 83fe27f749a34cb413a2cc34f05ed28cd265224966579560ced8472951e9528d
Transaction 372 committed: 44be0a63bbeed3d9ffc98e70134770f0a53703b3fe6d02f9633b7f3701da02af
Transaction 373 committed: 8721a3fc7e32ef7f2ffb0a47020d7abd584ca77dd40e9f25465f8

Transaction 455 committed: a761ed7a225d849df4fbffa3c52ec1cc549420703113f756548ce0449b993f8f
Transaction 456 committed: 563b051055f40a0dc6f8377c90312e54b480920e1be99d0ff7c7e6545231a2ba
Transaction 457 committed: 32428e30f6de856127bbed29fe709a4e6f040ef91e93179447db7b9aa6622a67
Transaction 458 committed: e7ff4976220d897cb12cd4ee1cbf0fde6a305a5355ebd7573be6d007e65d3f64
Transaction 459 committed: bdbfc5010797b0be337563b94a4b437724599e904fa19cd9ca7f2ac5b5565d55
Transaction 460 committed: 270188b7ceb6489717a243e1a3788de28102df178a04094e0dfce2434a76d612
Transaction 461 committed: 78a69e7667e8861c4f4bb0f75b4af966b5f293c6abe2dd013a51e1b6ffe58aa9
Transaction 462 committed: f47c2c932cc074965d7716b92107ebcdad0f07de77c906a40a71d843ea53c0b1
Transaction 463 committed: 9e3a2d3508a16fc183fbb62980507f20f929b74ff0f698e78465999ced503cca
Transaction 464 committed: 114b0f20228ba7cd5bfe0f1360abffeb635f260c4ca1100eb4e6e887762e951e
Transaction 465 committed: 68b8fc0096dfa99644b4f09bd1035f145c18255340db529b2dc95

In [30]:
def query_transactions_by_metadata(miner_address):
    start_time = time.time()
    transactions = bdb.transactions.get(asset_id='7e3491d79a1191ee9f2d4599fdfad74fb8721592c9213894900a8671ae82b045', operation='CREATE')
    matching_transactions = [tx for tx in transactions if tx['metadata'].get('miner') == miner_address]
    end_time = time.time()
    print(f"Found {len(matching_transactions)} transactions for miner {miner_address} in {end_time - start_time} seconds.")


query_transactions_by_metadata('0xe6a7a1d47ff21b6321162aea7c6cb457d5476bca')


Found 0 transactions for miner 0xe6a7a1d47ff21b6321162aea7c6cb457d5476bca in 0.06218290328979492 seconds.


In [31]:
transaction_ids = [596babc4d83bf6f9f8545dd10a087e8e58ab8374be88e339620bc53404ef7f48
f048e763084815b412565dca99f691d81d56cac2b1d2bfa14730357c32335704
3515672590baa20f98e05e2a471a0bd84021bea4577af9f92bf5cc9ff9f96535
c17d867163403076c077f84c3c5a596df256cd0aa6c20c9387fac659cb17cfb5
Transaction 5 committed: 46b1a175aaaff5f38d930300d735987058736c04797c22f7718ede3e61ad14f4
Transaction 6 committed: e525defa6bdd3d2f8e990f19df5a383bdfb1b9527fd8c9b86666879edc43ed72
Transaction 7 committed: 9d79343be182c576d1ccb8f089a84092dbf08dca29c29e1f4b5a0848d30f0687
Transaction 8 committed: 80296c8af856d4937c73395223917ec650f148b6cede6ac185c012f832ba9f84
Transaction 9 committed: b26a222c4370cdfb412610cb5256f78bb2778c239b92bb29ac4c8ccefc315bf9]

def query_transaction_by_id(transaction_id):
    start_time = time.time()
    try:
        transaction = bdb.transactions.retrieve(transaction_id)
        print(f"Transaction {transaction_id} retrieved successfully.")
    except Exception as e:
        print(f"Error retrieving transaction {transaction_id}: {e}")
    end_time = time.time()
    print(f"Query Time: {end_time - start_time} seconds")

sample_transaction_id = '7e3491d79a1191ee9f2d4599fdfad74fb8721592c9213894900a8671ae82b045'
query_transaction_by_id(sample_transaction_id)


Transaction 7e3491d79a1191ee9f2d4599fdfad74fb8721592c9213894900a8671ae82b045 retrieved successfully.
Query Time: 0.0574798583984375 seconds


[]


In [45]:
import s3fs
import pandas as pd
import psutil
import time
from datetime import datetime, timedelta
from bigchaindb_driver import BigchainDB
from bigchaindb_driver.crypto import generate_keypair

# BigchainDB setup
bdb = BigchainDB('http://localhost:9984')
user = generate_keypair()

# S3 setup
fs = s3fs.S3FileSystem(anon=True)
base_directory_path = 's3://aws-public-blockchain/v1.0/eth/blocks/date='

def log_system_resources():
    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().percent
    print(f"CPU Usage: {cpu_usage}%, Memory Usage: {memory_usage}%")

def load_data_into_bigchaindb(dataframe):
    # Ensure numeric data is kept as such, and convert dates to ISO 8601 format strings
    dataframe = dataframe.apply(lambda x: x if not x.name in dataframe.select_dtypes(include=['datetime64']) else x.dt.strftime('%Y-%m-%dT%H:%M:%S'))
    
    # Replace NaN, Infinity, and -Infinity with None
    dataframe.replace([float('inf'), float('-inf'), pd.NA, pd.NaT], None, inplace=True)

    for index, row in dataframe.iterrows():
        asset_data = {'hash': row['hash']}
        metadata = row.drop('hash').to_dict()

        # Convert non-string data explicitly to ensure they are stored correctly in BigchainDB
        for key, value in metadata.items():
            if pd.isna(value):
                metadata[key] = None
            elif isinstance(value, (int, float, bool)):
                metadata[key] = value
            else:
                metadata[key] = str(value)
        
        prepared_tx = bdb.transactions.prepare(
            operation='CREATE',
            signers=user.public_key,
            asset={'data': asset_data},
            metadata=metadata,
        )
        fulfilled_tx = bdb.transactions.fulfill(prepared_tx, private_keys=user.private_key)
        sent_tx = bdb.transactions.send_commit(fulfilled_tx)
        print(f"Transaction {index+1} committed: {sent_tx['id']}")

def process_date_range(start_date, end_date):
    start_time = time.time()
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime('%Y-%m-%d')
        directory_path = f'{base_directory_path}{date_str}'
        try:
            files = fs.ls(directory_path)
            parquet_files = [file for file in files if file.endswith('.snappy.parquet')]
            for file_path in parquet_files:
                with fs.open(f's3://{file_path}', mode='rb') as f:
                    df = pd.read_parquet(f, engine='pyarrow')
                    load_data_into_bigchaindb(df)
        except Exception as e:
            print(f"Error processing {date_str}: {e}")
        current_date += timedelta(days=1)
        log_system_resources()

    end_time = time.time()
    print(f"Total time for processing range: {end_time - start_time} seconds")

start_date = datetime(2016, 7, 30)
process_date_range(start_date, start_date + timedelta(days=0))  # Adjust the range as needed


Transaction 1 committed: d89af8a0d7c0fa29ce67e16bb929a78243209162de92a3f60d1166ee21b415d1
Transaction 2 committed: 8b7c958d30774af71ad1c2643a29e69380e617a6e23341f21df84b062a4ddeae
Transaction 3 committed: 5ee637957d54cb15a92164d8e5dbf049c07eaee0ba188f119602efd5a7e0771a
Transaction 4 committed: 3b7e2d0b5c78274ba31d50a846f2ab81ad9133f51e3a11b60c1265b5152b2c81
Transaction 5 committed: ccecb2fcbc35d03b895fb5753c064fa777bc3203aabfa7a350109f5b8b2cdf33
Transaction 6 committed: a9f0c0a9e7575193c1d9e8294a18b6f2b36f33ec651cf4fbc87c055840f32067
Transaction 7 committed: f23e6f3f1ada497565ef674cd105e65a5b6cfeab5676160c212a880e58ae402d
Transaction 8 committed: 44b3ea4daac763805e9250643aeb693a8f75c5201ee2ff64d2dfbcfa1eb88532
Transaction 9 committed: 620e5335fc5f03c592e1aa1e4662300c05e821a9f23321df2f6f45ab2b81a278
Transaction 10 committed: 8e8b6e60fdd16de7b9ecf893b861b4b5724011a2f9441f1651fa4c3869c7b537
Transaction 11 committed: 3879480551fd785a58787a44b9a4ff2b33a91eb96e8456976d3879634c691dfc
Transact

Transaction 92 committed: cfdaf1247e84ae52ade0040069718131ba689d0e89a326f2f18bfb141119c819
Transaction 93 committed: 2dcf41ce228ae38ffb733a73eb5276bb2a381d4ee7b338bfb392729e22b7b925
Transaction 94 committed: 428a3a098a38e2428eef4bd5993162d871727bcbaac036fa32a0d5de370d3571
Transaction 95 committed: 2337287e0db3aaf92ed7ce0eeb82e9160e39ac549258ce8f96cff2bd2418ad69
Transaction 96 committed: 4956aa68eb2965732d7cab771584aba5ce9424cd57c127ee8511fc9102d845b2
Transaction 97 committed: 731370bf41ff77f5c59a29724b309fb1b9b761048b4730424c1b725d611e6b80
Transaction 98 committed: d704661f4fddf14e2758deb9f53ac1135728a27e50bf0e675b336cfc74c001cd
Transaction 99 committed: 8572704b568efef8456c3c175440a771753a9d26a509dffffdbcbfe7ff50310a
Transaction 100 committed: 9a084b135686a170f5f9f7c85c61b97fb0c349c2631b0b458f4bac9fb21cf609
Transaction 101 committed: 0db062b9b7a07b45a45f396619cf851448cec75328e95dd3aa4c912420c19853
Transaction 102 committed: fac6bc8a3afd2b6cb137dcbe10ca2b371c38ea067cad4baabcdf63aaeb486

Transaction 184 committed: 3c3b784341768eeafe66efa4303a7a6b5163f5c9feb5a669ab4b0b9d3193b8dc
Transaction 185 committed: d88a279c7136620fad96a1a9bdaa76bf9e7760ac93430d7c3fe345ebe96117ee
Transaction 186 committed: 6603e16a659c729e2ff1acce9e77d5a2a03378d658a3906200d2765648c89db5
Transaction 187 committed: 029f392ae4ebbc7c43488cd08590fa0bb4565c2bdfc34ecdeb5c68eac45eff27
Transaction 188 committed: bc35dab0ec737ab338bf8e9c23c24814684507cddfbd52f452ea4cc0421ef6db
Transaction 189 committed: b7ca1769f671f92bdf92da1ffff8154d4ee7befd2f10a312184890aff0375e32
Transaction 190 committed: 2d36f549875a3a92cc4d2e1e4399d7bb97d63f406363e5c133352ca7ef84b291
Transaction 191 committed: 6de4188da5a1a45b9fbb8d36d0461e28ce359ef81252300473f479e3359b02f1
Transaction 192 committed: ba03de1198c3adcf2c403ddca3954a9480614d95108a240a70cf9df278ac3fa0
Transaction 193 committed: fb801259cbd97e561520fa388375e8e9cd266d87870a4c513fe4772a5c4899c7
Transaction 194 committed: 173eecdfd469b7a9a50f23355fdac548a58745014b872391ccb3b

Transaction 274 committed: c13b5c06ead9ff86931f7e100877bb93942d2931e12a832d9ba889c08230aa11
Transaction 275 committed: 8b4a8967989879e6906475ba21c7f97a0cf15682f6abf1354a762b7d5d68fd56
Transaction 276 committed: d32230a8aab5e785f1a509baaf38dcd13917fd2d618ba375b60060e8b9b813f3
Transaction 277 committed: 9a7c46400d00912708f02ca665a31c8db3a1a33cf0f4370da19c08c0b0ac8245
Transaction 278 committed: 2e7ecbbd3d12e3d975a3b8f4f705d960b85e58419b4cc07b5cc87bbfde8e6452
Transaction 279 committed: f565532ec3de4bcbe7c9c481196f6c7fd5bf9c456deac2fe6959aa26ec6bfcea
Transaction 280 committed: 01d73a73ac990ac9ef5bebfc8a889b41a5c136be7bb63e6b5541c0905554d399
Transaction 281 committed: 3ecc985e20d57b83ec265a8b3adcd3bac9f671555be9eaf9460f6af15c29dc2c
Transaction 282 committed: ac4b631d6a54ee4d7ee92f5a2a7c8475e0f6f47aea2931a02af4769c012894a6
Transaction 283 committed: 123290b4183cd6517e7e5bc2d4651d6f4e78741062dd6cd35b3d56a6e135e2f6
Transaction 284 committed: 2b1b023c32845713521aae9a30797998c0fbc69a0c41f7931a95b

Transaction 365 committed: 2b62403e310abce218b39fcf1c983d55721df2d5f2281fa42f1559b43ff076c2
Transaction 366 committed: f5381727d2a99150f32cac6692f4a8d875a467c02dd8624c968ae76ffc19c5fe
Transaction 367 committed: 9f92195ef5a449037775fd551c512d35352472244ccf93d14c8b45cacdc97a7c
Transaction 368 committed: 15b2697033dd58edeedd15fd826bc4c4e9c9d4fb372210aa39fb614ff20ee01d
Transaction 369 committed: 77eb14d09c1d2278fc2e6d21f9a08e3a5aa1f0ed0d0134c431eea05b0e3f3ed9
Transaction 370 committed: 2db0f2528637726e25b65031228fe787aa950e0f059355aedfdcd7dd2fe5f710
Transaction 371 committed: 281624130fd41506b665f4e276ec859693acf74287a09e1ce3321a46f1bc36d5
Transaction 372 committed: c4151dfecb2c2dd2c52b99a1f2a15a73876773fe9359e5ed164aac4110b845f3
Transaction 373 committed: 46112929a46b390be704fb2a40618690de04749baeb4a3c2afa5a54053751aa4
Transaction 374 committed: 735b62d98fc328b2d5f00ec0066ab5475e557a0a79d088f201186a467cfb19c3
Transaction 375 committed: c166df4c18dac89f5a0f2754bf1d4b1a97c23a945448ae6ce87ab

...

Transaction 5601 committed: 92d2762ff05cdd3edb2691d4ba07e77c0c52f081ffffe836c139261f0fe37eaa
Transaction 5602 committed: bd92ae782096746ddca9c35f8d0ddc6b51c589c8f85244857b938d972b58ee88
Transaction 5603 committed: 11be6e8e23146b74b78e46361f803fee7ca8dae01ecd5d5ce2689f826eed2801
Transaction 5604 committed: 0e03b6302324885adcd916ca34389aae5fd60a562ff08a2729cc3c5f4c65c34a
Transaction 5605 committed: b85c785b975a83f63af89ba584272c7c81c8962ad16e2794a36bbf8ba0bac07b
Transaction 5606 committed: 45c91cc66677caddac862d2863abc5fd1a40792a5f45d252761075675a7e6279
Transaction 5607 committed: ed00d739b942d2d27d2837069254cf3d6b2cc8eec113bec63eacb6efb3748401
Transaction 5608 committed: 02638835a0211df992d3aeee03789f4986ec09d89660629ba360384a8eb557f2
Transaction 5609 committed: f46cce667ea7868a8493f0c2a842550077abb0d6e705e68e83d2c94f5162f6a3
Transaction 5610 committed: 420c745860e5cf14503b29b81d04dffe13065f9d7b3e392d59e53592f05b28b9
Transaction 5611 committed: 2e8541c2935e72aa30a03607e56795b9abde12c04b

Transaction 5690 committed: 37418e519d9d0e7b3e178b2c9491b9e8044d28f0907889b3ebcb4965f8f40187
Transaction 5691 committed: 2719be656caf8d64c0e668c64af904da2206da3f39e151bc557d229c3d3d9f54
Transaction 5692 committed: 4d699b0cb5fca6e5e4d238406df6c0fdc61f702b8933559a813f18961d058ad1
Transaction 5693 committed: c772d1632dbe41c8f4cebf0cb3446ec762e3330e8a022b7bf8ebcff6b6c2dcf4
Transaction 5694 committed: 81df15cf51092cd7644ac14b9437c1a1cda3ccdc1853ecf15ac45bebe10caab9
Transaction 5695 committed: d0a20a410fc42e096b09fdbdb9506819a108819fb1e2d6367c00ed0797923410
Transaction 5696 committed: efb50c9c5e41086e440f33eca15188cdf1a40cd68924eae8282de6ab81a28d0c
Transaction 5697 committed: c9195cbc4b85dacf38db312140d0cc0baab41284408a4bbef0f13095270d81f6
Transaction 5698 committed: f563c4f0870b61689d834e3fe7cae8069a5461323b7ff16c7ee0f8c91d85b721
Transaction 5699 committed: 47139a01600fd0e0961eee9384031e396d5098da347127f888780ccfc6d3721f
Transaction 5700 committed: ea5e54d3792db607ef9c4fc5f85afac73231e71707

Transaction 5779 committed: 8c6b59bee489ee6e2c8713d405208c73026756c35c4044e7f02e469ea93d927c
Transaction 5780 committed: dfe30f74d7993fcd51f258f3def117107e6b83ec622e20084c53474b2a2e2335
Transaction 5781 committed: 50731f0736a4ef8c693800ce162608fe370e7c44fd8fde6bfdf834e5bf00e3b0
Transaction 5782 committed: efeb6639b3eb6a94b4a29b88585d2b0198061e3925818e449903b814736d4f11
Transaction 5783 committed: 1aef450b67132979bd2d6e26bae5689b0098c28cc322475b06417ae3d123bf63
Transaction 5784 committed: 4c9aaa060239cf34dffab9214a547dca280f3f64afb96c73f16e972dfd029d02
Transaction 5785 committed: 5d5660fc2b2794b0307342fe68686e806a06280e2a988d83f5710ab4fa73423f
Transaction 5786 committed: 119c8db4c2aea3a62ee1518501e823ad25d60c4f5418415b4b62fbcaedaab7b3
Transaction 5787 committed: cf4237a2fd1f6f7dd292ad8cdb8bd9462fe82759870c009fcb271da680639097
Transaction 5788 committed: 1ca4a20aba98afef5ff74a2a3eaa38e2f52b41f0931f9bb5d7b6016cc0a1aaec
Transaction 5789 committed: 64a01bf9c2d92fe9a0b8e68dbef673c633be3ac0c5

Transaction 5870 committed: 6045d9956bc748a8e12a5b0a480c2d843740156f975e831e1e24f688c9d03465
Transaction 5871 committed: 2db4ac807c4685aaa51a31eaaf2122f9b45eecb9bfb5e04903e55914d37daa1f
Transaction 5872 committed: d3632184fe8d98e0b77bcf1c6518eef1dddc2ead657dc1e0b316e1a1d5b43378
Transaction 5873 committed: 203c9b0a07aa7f9fd68d692ed2ee796e7022fc97eaa0145acb3da29ae770d37b
Transaction 5874 committed: 44383a44ebc78aa127678e154ad74219521997543433568c394acefa61bb60b8
Transaction 5875 committed: 2f1643d998a0826699fd99aa28bcb3f9e43ab6efbcbc1de573182b9f15efb958
Transaction 5876 committed: 3a57d8639b0d45961f2a01a0e04dc35bb959cafaee5dac96d20749c1906bf41f
Transaction 5877 committed: e275225ef04cb6ea012a1a627e823299236c170a9a6b52135ad4a035be0df472
Transaction 5878 committed: 15e0d60ac65812776b67428b68eb996bef8b764151b8f95ae41b0e31f30e1bba
Transaction 5879 committed: 549b47f6c2afc119fb41383a1ff687a2d2cd23175b248543992ddfaa397042ee
Transaction 5880 committed: 07d281f05bad62b0255877ca9716f2a8dc6fc17f05

Transaction 5960 committed: 7adebd9d688738cb5fc0f246ad9bc1359805abc3c92a0f563effaafa621f88cc
Transaction 5961 committed: b8433de114e79f78ad318c1368cc3049d94a927c247a2fd7abf203aaa2dc3485
Transaction 5962 committed: 8e4a54583f6b3684fce54bf4c332729394ee6c78a49555031678f30f878854bd
Transaction 5963 committed: d08324997eaf17844b9ea33ff1842078323ce008c49d2baac7f01ff10fa2d299
Transaction 5964 committed: 64dba94f1a8d89941d8b6438198f4666eb77030fc97a4964c522879221cc293e
Transaction 5965 committed: b37cce29f60dd3805fd0085504619246e89bdb4d5e456c7f943d597352effd08
Transaction 5966 committed: 3b3193ca4d8ec6c0c1d728080257317f7b4b706625fbd35a37fe906797bfe47f
Transaction 5967 committed: da1500b3dd6b0d761d94eff97e852d3858cb85b0e7172ce00e2e0959a97835f4
Transaction 5968 committed: b600cd99827bb2beb90a13fcf4c23f3721c6b32c3b1c42584c06943b6ca209b6
Transaction 5969 committed: 9f0056a259cec4d351c6318c1c6ea4cd43bda391525e8461650ae4b28a8ca372
Transaction 5970 committed: 5eb251aa379559e4211eaf88c5c8d4d033ea752550

In [49]:
miner_address = '0x7a1458f33122080819a2483f2a16034cd8cb0b95'
search_results = bdb.metadata.get(search='0x7a1458f33122080819a2483f2a16034cd8cb0b95')

# Filter results in application code
# filtered_results = [result for result in search_results if 'gas_limit' in result['metadata'] and result['metadata']['gas_limit'] == 351153]
# print(filtered_results)
print(search_results)


[{'id': '5ac301d29d05a7b498bb8cf6ad3c661546098f85415108eabe7c94096d635d7b', 'metadata': {'difficulty': 54443816312321.0, 'miner': '0x7a1458f33122080819a2483f2a16034cd8cb0b95', 'nonce': '0xf0d62200043f1861', 'number': 1977679, 'size': 539, 'timestamp': '2016-07-30T03:51:30', 'total_difficulty': 4.281250626523202e+19, 'base_fee_per_gas': None, 'gas_limit': 4700000, 'gas_used': 0, 'extra_data': '0xd5830102028650617269747986312e31302e30826c69', 'logs_bloom': '0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000', 'parent_hash': '0x22efa

In [51]:
# Given search_results from bdb.metadata.get
filtered_results = [result for result in search_results if result['metadata'].get('gas_limit') >= 4707788]
print(filtered_results)


[{'id': 'c9484ab50f29627fd28354928c8a0b5b73d79bc709bda777af0a2e24a6e6c5a7', 'metadata': {'difficulty': 54414366006658.0, 'miner': '0x7a1458f33122080819a2483f2a16034cd8cb0b95', 'nonce': '0x34958f6c0dbedbaa', 'number': 1977905, 'size': 539, 'timestamp': '2016-07-30T04:45:35', 'total_difficulty': 4.2824812850361156e+19, 'base_fee_per_gas': None, 'gas_limit': 4707788, 'gas_used': 0, 'extra_data': '0xd5830102028650617269747986312e31302e30826c69', 'logs_bloom': '0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000', 'parent_hash': '0x81c9

In [52]:
import pandas as pd
import s3fs
from bigchaindb_driver import BigchainDB
from bigchaindb_driver.crypto import generate_keypair

# # Function to load data from a Parquet file and process transactions
# def load_data_from_parquet(file_path, bdb, user):
#     with fs.open(file_path, mode='rb') as f:
#         df = pd.read_parquet(f, engine='pyarrow')

#     df['block_timestamp'] = df['block_timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')

#     for index, row in df.iterrows():
#         asset_data = {
#             'token_address': row['token_address'],
#             'from_address': row['from_address'],
#             'to_address': row['to_address'],
#             'value': row['value'],
#             'transaction_hash': row['transaction_hash'],
#             'log_index': row['log_index'],
#             'block_number': row['block_number'],
#             'block_hash': row['block_hash'],
#             'block_timestamp': row['block_timestamp'],  # Now a string
#         }

#         prepared_tx = bdb.transactions.prepare(
#             operation='CREATE',
#             signers=user.public_key,
#             asset={'data': asset_data},
#         )

#         fulfilled_tx = bdb.transactions.fulfill(
#             prepared_tx,
#             private_keys=user.private_key,
#         )

#         sent_tx = bdb.transactions.send_commit(fulfilled_tx)
#         print(f"Transaction {index+1} committed: {sent_tx['id']}")

# Initializing BigchainDB and S3FileSystem
bdb = BigchainDB('http://localhost:9984')
user = generate_keypair()
fs = s3fs.S3FileSystem(anon=True)

# Directories to process
directories = [
    'aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-27',
    'aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-30',
    'aws-public-blockchain/v1.0/eth/token_transfers/date=2015-11-02',
    'aws-public-blockchain/v1.0/eth/token_transfers/date=2015-11-03',
    'aws-public-blockchain/v1.0/eth/token_transfers/date=2015-11-04',
]

# Function to get file sizes in MB
def get_file_sizes_in_mb(directory_path):
    files = fs.ls(directory_path)
    parquet_files = [file for file in files if file.endswith('.parquet')]
    
    total_size_mb = 0
    for file_path in parquet_files:
        file_info = fs.info(file_path)
        file_size_mb = file_info['size'] / (1024 * 1024)  # Convert bytes to MB
        total_size_mb += file_size_mb
        print(f"File: {file_path}, Size: {file_size_mb:.2f} MB")
    print(f"Total size for {directory_path}: {total_size_mb:.2f} MB")

# Processing each directory to get file sizes
for directory_path in directories:
    print(f"Processing directory: {directory_path}")
    get_file_sizes_in_mb(directory_path)

print("All transactions from specified dates have been processed and file sizes calculated.")


Processing directory: aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-27
File: aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-27/part-00000-2ad0be30-0be0-4d1a-8d1f-c7240a656db5-c000.snappy.parquet, Size: 0.01 MB
Total size for aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-27: 0.01 MB
Processing directory: aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-30
File: aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-30/part-00000-7996c555-d36b-4163-b792-77c5afe1a0b5-c000.snappy.parquet, Size: 0.01 MB
Total size for aws-public-blockchain/v1.0/eth/token_transfers/date=2015-10-30: 0.01 MB
Processing directory: aws-public-blockchain/v1.0/eth/token_transfers/date=2015-11-02
File: aws-public-blockchain/v1.0/eth/token_transfers/date=2015-11-02/part-00000-c318a2d3-0ddb-4f31-b39f-284e6e47c025-c000.snappy.parquet, Size: 0.01 MB
Total size for aws-public-blockchain/v1.0/eth/token_transfers/date=2015-11-02: 0.01 MB
Processing directory: aws-pu

In [ ]:
#Update Transactions
transaction_id = '1391d6c7c17e464b6fd3255b61e08c59ebdbd13536574538de5cafec50c40620'

# Fetch the transaction
transaction = bdb.transactions.retrieve(transaction_id)

# Generate keypair for the sake of example
alice = generate_keypair()

# Assuming the 'asset' field contains the data we want to update
asset_id = transaction['id']
new_asset = {
    'data': {
        'transaction_data': transaction['asset']['data']['transaction_data'],
        'gas': 'new_gas_value'  # Update the gas field here
    }
}

# Prepare the transaction with the new asset
prepared_tx = bdb.transactions.prepare(
    operation='TRANSFER',
    asset={'id': asset_id},
    recipients=alice.public_key,
    metadata={'update': 'Updating gas field'}
)

# Sign the transaction with Alice's private key
fulfilled_tx = bdb.transactions.fulfill(
    prepared_tx,
    private_keys=alice.private_key
)

# Send the transaction
sent_tx = bdb.transactions.send_commit(fulfilled_tx)

print('Transaction', sent_tx['id'], 'has been posted.')


In [ ]:
#Delete the transactions.
transaction_id = '1391d6c7c17e464b6fd3255b61e08c59ebdbd13536574538de5cafec50c40620'

# Fetch the transaction
transaction = bdb.transactions.retrieve(transaction_id)


alice = generate_keypair()

# Use the asset ID from the existing transaction
asset_id = transaction['id']

prepared_tx = bdb.transactions.prepare(
    operation='TRANSFER',
    asset={'id': asset_id},
    recipients=None,  # No recipient, symbolizes deletion
    metadata={'status': 'deleted', 'reason': 'No longer needed'}
)

fulfilled_tx = bdb.transactions.fulfill(
    prepared_tx,
    private_keys=alice.private_key
)

# Send the transaction
sent_tx = bdb.transactions.send_commit(fulfilled_tx)

print('Transaction', sent_tx['id'], 'has been marked as deleted.')


In [ ]:
#Node Synchornization 

import time
from pymongo import MongoClient

# Connection to MongoDB used by BigchainDB
client = MongoClient('mongodb://localhost:27017/')
db = client.bigchain

def get_block_height():
    """ Retrieve the current block height from BigchainDB. """
    return db.blocks.count_documents({})

def is_synced(latest_known_block, current_block_height):
    """ Check if the node has reached the latest known block height. """
    return current_block_height >= latest_known_block

def synchronize_node(latest_known_block):
    """ Simulate the node synchronization process. """
    start_time = time.time()
    current_block_height = get_block_height()

    while not is_synced(latest_known_block, current_block_height):
        time.sleep(1)  # Wait for a while before checking again
        current_block_height = get_block_height()

    end_time = time.time()
    return end_time - start_time

latest_known_block = 50000  # Note:  This should be set according to the current state of the blockchain

# Measure synchronization time
sync_time = synchronize_node(latest_known_block)
print(f"Node synchronization took {sync_time} seconds.")
